In [1]:
import pandas as pd

In [2]:
# what we want is to read a lgofile with multiple logs in it

df = pd.read_csv('data/21-jun-22/TESTLOG_20220622.dtf',
                 header=216,
                 parse_dates=True,
                 dayfirst=True,
                 infer_datetime_format=True,
                )

In [3]:
df

,Sample #,UTC Date,UTC Time,GPS_Fix,Lat,Lon,Solar Azi,Solar Elev,Relaz,Heading,...,Spec[211],Spec[212],Spec[213],Spec[214],Spec[215],Spec[216],Spec[217],Spec[218],Spec[219],Spec[220]
0,OUTPUT LOG_0063.TXT DATA TO SCREEN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FILESIZE (Bytes)=180563.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,In-situ Marine Optics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DALEC (SN:0005),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FIRMWARE: DALEC_V5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,39,21/06/2022,09:31:07.030,A,56.0810852,-3.99958496,120.8,46.3,85.5,206.3,...,0.082830,0.081549,0.082379,0.078895,0.080422,0.075147,0.074971,0.075330,0.073814,0.074468
425,40,21/06/2022,09:31:09.034,A,56.0803718,-3.99969673,120.8,46.3,85.8,206.6,...,0.169215,0.164241,0.160928,0.160912,0.160965,0.156963,0.151662,0.145298,0.145111,0.138846
426,40,21/06/2022,09:31:09.034,A,56.0803718,-3.99969673,120.8,46.3,85.8,206.6,...,0.001824,0.001803,0.001717,0.001706,0.001689,0.001686,0.001616,0.001545,0.001487,0.001535
427,40,21/06/2022,09:31:09.034,A,56.0803718,-3.99969673,120.8,46.3,85.8,206.6,...,0.084119,0.083355,0.082970,0.077099,0.080059,0.078177,0.072079,0.073553,0.074060,0.078654


In [16]:
df['Sample #']

# here we're choosing a line to delineate between different log files
# perhaps could generalise this with 'END OF FILE' or something (a task for another day!)

table_names = ['OUTPUT LOG_0063.TXT DATA TO SCREEN',
               'OUTPUT LOG_0062.TXT DATA TO SCREEN',
               'OUTPUT LOG_0061.TXT DATA TO SCREEN']

groups = df['Sample #'].isin(table_names).cumsum()
# produces a dictionary with keys as the table names and values as the subtables.
# see https://stackoverflow.com/questions/34184841/python-pandas-read-csv-file-containing-multiple-tables
tables = {g.iloc[0,0]: g.iloc[1:] for k,g in df.groupby(groups)} 


In [52]:
# let's try to generalise this

groups = df['Sample #'].isin(['DALEC (SN:0005)']).cumsum()
names = ['Log ' + str(i) for i in range(len(set(groups)))] 
tables = {name: g[1].iloc[1:] for g,name in zip(df.groupby(groups), names)} 
# because we've used 'DALEC (SN:0005)' as the way to seperate logfiles, we need to remove the first lines before this
tables.pop('Log 0')

,Sample #,UTC Date,UTC Time,GPS_Fix,Lat,Lon,Solar Azi,Solar Elev,Relaz,Heading,...,Spec[211],Spec[212],Spec[213],Spec[214],Spec[215],Spec[216],Spec[217],Spec[218],Spec[219],Spec[220]
1,FILESIZE (Bytes)=180563.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,In-situ Marine Optics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
list(tables)

for k,v in tables.items():
    print("table:", k)
    print(v)
    print()

table: Log 0
                   spectral_ind  Heading  Pitch  Relaz         Lat  Gearpos  \
Sample #  Channel                                                             
0        Ed                  21      0.0    0.0    NaN         NaN     -0.1   
         Ed                  22      0.0    0.0    NaN         NaN     -0.1   
         Ed                  23      0.0    0.0    NaN         NaN     -0.1   
         Ed                  24      0.0    0.0    NaN         NaN     -0.1   
         Ed                  25      0.0    0.0    NaN         NaN     -0.1   
...                         ...      ...    ...    ...         ...      ...   
40       Lu                 216    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 217    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 218    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 219    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 220    204.

In [23]:
# okay now, let's remove some rows which aren't actually data
df.drop(df[df[' UTC Date'].isna()].index, inplace = True)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [53]:
for name, table in tables.items():
    # any row with invalid UTC date can be removed
    table.drop(table[table[' UTC Date'].isna()].index, inplace = True)
    # this removes the duplicated headings
    table.drop(table[table[' UTC Date'] == 'UTC Date'].index, inplace = True)
    
    # convert to long format
    # need to test that these variable names always load in this way (leading space on Spec etc.)
    table = pd.wide_to_long(table, [' Spec['], i=['Sample #', ' Channel'], j='spectral_ind', suffix='\d+]')
    table.reset_index(level=2, inplace=True) # remove spectral_ind as an index
    table['spectral_ind'] = pd.to_numeric(table['spectral_ind'].str[:-1]) # convert spectral_ind to numeric
    table.rename(columns={' Spec[': 'Spectral Magnitude'}, inplace=True)
    table = table.astype({'Spectral Magnitude': 'float64'})
    # change sample no. index to integer
    print('WARNING: some of my old code wont work with integerIndex=True - delete this line once this is sorted')
    idx = table.index
    table.index = table.index.set_levels([idx.levels[0].astype(int), idx.levels[1]])
    # sort index
    table.sort_index(inplace=True)
    tables[name] = table

In [26]:
# check to see if that worked
for k,v in tables.items():
    print("table:", k)
    print(v)
    print()

table: Log 1
                   spectral_ind  Heading  Pitch  Relaz         Lat  Gearpos  \
Sample #  Channel                                                             
0        Ed                  21      0.0    0.0    NaN         NaN     -0.1   
         Ed                  22      0.0    0.0    NaN         NaN     -0.1   
         Ed                  23      0.0    0.0    NaN         NaN     -0.1   
         Ed                  24      0.0    0.0    NaN         NaN     -0.1   
         Ed                  25      0.0    0.0    NaN         NaN     -0.1   
...                         ...      ...    ...    ...         ...      ...   
40       Lu                 216    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 217    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 218    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 219    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 220    204.

In [54]:
# now let's try to sort out the saturated values
# we want to remove any sample which has a saturated value

exTable = tables['Log 1']

ind = exTable[exTable[' Saturation Flag'] == '1'].index.get_level_values(0)
exTable.drop(ind, level=0, axis=0, inplace=True)
exTable 

spectral_ind  Heading  Pitch  Relaz         Lat  Gearpos  \
Sample #  Channel                                                             
5        Ed                  21    205.7    5.0    NaN         NaN      0.0   
         Ed                  22    205.7    5.0    NaN         NaN      0.0   
         Ed                  23    205.7    5.0    NaN         NaN      0.0   
         Ed                  24    205.7    5.0    NaN         NaN      0.0   
         Ed                  25    205.7    5.0    NaN         NaN      0.0   
...                         ...      ...    ...    ...         ...      ...   
40       Lu                 216    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 217    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 218    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 219    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 220    204.9    5.0   25.3  56.0295791      0.0   

                   Integration Time     Temp  Saturation Flag          Lon  \
Sample #  Channel                                                            
5        Ed                      42   21.375                0          NaN   
         Ed                      42   21.375                0          NaN   
         Ed                      42   21.375                0          NaN   
         Ed                      42   21.375                0          NaN   
         Ed                      42   21.375                0          NaN   
...                             ...      ...              ...          ...   
40       Lu                     256  21.6875                0  -4.06032180   
         Lu                     256  21.6875                0  -4.06032180   
         Lu                     256  21.6875                0  -4.06032180   
         Lu                     256  21.6875                0  -4.06032180   
         Lu                     256  21.6875                0  -4.06032180   

                   Solar Azi  Roll  Solar Elev    UTC Date  GPS_Fix  Voltage  \
Sample #  Channel                                                              
5        Ed              NaN   1.0         NaN  21/06/2022        V     11.5   
         Ed              NaN   1.0         NaN  21/06/2022        V     11.5   
         Ed              NaN   1.0         NaN  21/06/2022        V     11.5   
         Ed              NaN   1.0         NaN  21/06/2022        V     11.5   
         Ed              NaN   1.0         NaN  21/06/2022        V     11.5   
...                      ...   ...         ...         ...      ...      ...   
40       Lu            179.6   0.0        57.7  21/06/2022        A     11.5   
         Lu            179.6   0.0        57.7  21/06/2022        A     11.5   
         Lu            179.6   0.0        57.7  21/06/2022        A     11.5   
         Lu            179.6   0.0        57.7  21/06/2022        A     11.5   
         Lu            179.6   0.0        57.7  21/06/2022        A     11.5   

                       UTC Time  Spectral Magnitude  
Sample #  Channel                                    
5        Ed        12:10:50.108            0.231670  
         Ed        12:10:50.108            0.237858  
         Ed        12:10:50.108            0.236359  
         Ed        12:10:50.108            0.226115  
         Ed        12:10:50.108            0.227510  
...                         ...                 ...  
40       Lu        12:11:59.031            0.001691  
         Lu        12:11:59.031            0.001588  
         Lu        12:11:59.031            0.001711  
         Lu        12:11:59.031            0.001530  
         Lu        12:11:59.031            0.001499  

[21600 rows x 18 columns]

In [51]:
tables['Log 1']

spectral_ind  Heading  Pitch  Relaz         Lat  Gearpos  \
Sample #  Channel                                                             
5        Ed                  21    205.7    5.0    NaN         NaN      0.0   
         Ed                  22    205.7    5.0    NaN         NaN      0.0   
         Ed                  23    205.7    5.0    NaN         NaN      0.0   
         Ed                  24    205.7    5.0    NaN         NaN      0.0   
         Ed                  25    205.7    5.0    NaN         NaN      0.0   
...                         ...      ...    ...    ...         ...      ...   
40       Lu                 216    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 217    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 218    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 219    204.9    5.0   25.3  56.0295791      0.0   
         Lu                 220    204.9    5.0   25.3  56.0295791      0.0   

                   Integration Time     Temp  Saturation Flag          Lon  \
Sample #  Channel                                                            
5        Ed                      42   21.375                0          NaN   
         Ed                      42   21.375                0          NaN   
         Ed                      42   21.375                0          NaN   
         Ed                      42   21.375                0          NaN   
         Ed                      42   21.375                0          NaN   
...                             ...      ...              ...          ...   
40       Lu                     256  21.6875                0  -4.06032180   
         Lu                     256  21.6875                0  -4.06032180   
         Lu                     256  21.6875                0  -4.06032180   
         Lu                     256  21.6875                0  -4.06032180   
         Lu                     256  21.6875                0  -4.06032180   

                   Solar Azi  Roll  Solar Elev    UTC Date  GPS_Fix  Voltage  \
Sample #  Channel                                                              
5        Ed              NaN   1.0         NaN  21/06/2022        V     11.5   
         Ed              NaN   1.0         NaN  21/06/2022        V     11.5   
         Ed              NaN   1.0         NaN  21/06/2022        V     11.5   
         Ed              NaN   1.0         NaN  21/06/2022        V     11.5   
         Ed              NaN   1.0         NaN  21/06/2022        V     11.5   
...                      ...   ...         ...         ...      ...      ...   
40       Lu            179.6   0.0        57.7  21/06/2022        A     11.5   
         Lu            179.6   0.0        57.7  21/06/2022        A     11.5   
         Lu            179.6   0.0        57.7  21/06/2022        A     11.5   
         Lu            179.6   0.0        57.7  21/06/2022        A     11.5   
         Lu            179.6   0.0        57.7  21/06/2022        A     11.5   

                       UTC Time  Spectral Magnitude  
Sample #  Channel                                    
5        Ed        12:10:50.108            0.231670  
         Ed        12:10:50.108            0.237858  
         Ed        12:10:50.108            0.236359  
         Ed        12:10:50.108            0.226115  
         Ed        12:10:50.108            0.227510  
...                         ...                 ...  
40       Lu        12:11:59.031            0.001691  
         Lu        12:11:59.031            0.001588  
         Lu        12:11:59.031            0.001711  
         Lu        12:11:59.031            0.001530  
         Lu        12:11:59.031            0.001499  

[21600 rows x 18 columns]

In [57]:
pd.to_datetime(exTable[' UTC Time'], infer_datetime_format=True)

Sample #   Channel
5         Ed         2022-06-23 12:10:50.108
          Ed         2022-06-23 12:10:50.108
          Ed         2022-06-23 12:10:50.108
          Ed         2022-06-23 12:10:50.108
          Ed         2022-06-23 12:10:50.108
                               ...          
40        Lu         2022-06-23 12:11:59.031
          Lu         2022-06-23 12:11:59.031
          Lu         2022-06-23 12:11:59.031
          Lu         2022-06-23 12:11:59.031
          Lu         2022-06-23 12:11:59.031
Name:  UTC Time, Length: 21600, dtype: datetime64[ns]